In [1]:
# https://github.com/aladdinpersson/Machine-Learning-Collection/blob/master/ML/TensorFlow/Basics/tutorial9-custom-layers.py

import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import mnist

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
os.environ["TF_ENABLE_ONEDNN_OPTS"] = "1"
os.environ["MKL_DEBUG_CPU_TYPE"] = "5"
os.environ["KMP_BLOCKTIME"] = "1"
os.environ["KMP_AFFINITY"] = "granularity=fine,compact,1,0"
os.environ["OMP_NUM_THREADS"] = "<num_threads>"

gpus = tf.config.experimental.list_physical_devices("GPU")
if gpus:
      for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
            physical_devices = tf.config.list_physical_devices("GPU")
            tf.config.experimental.set_memory_growth(physical_devices[0], True)
else:
      print("No GPU device found")

No GPU device found


In [2]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(-1, 28 * 28).astype("float32") / 255.0
x_test = x_test.reshape(-1, 28 * 28).astype("float32") / 255.0

In [3]:
class Dense(layers.Layer):
    def __init__(self, units, input_dim):
        super(Dense, self).__init__()
        self.w = self.add_weight(
            name="w",
            shape=(input_dim, units),
            initializer="random_normal",
            trainable=True
        )
        self.b = self.add_weight(
            name="b",
            shape=(units,),
            initializer="zeros",
            trainable=True
        )
        
    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b

In [4]:
class Dense(layers.Layer):
    def __init__(self, units):
        super(Dense, self).__init__()
        self.units = units
        
    def build(self, input_shape):
        self.w = self.add_weight(
            name="w",
            shape=(input_shape[-1], self.units),
            initializer="random_normal",
            trainable=True
        )
        self.b = self.add_weight(
            name="b",
            shape=(self.units,),
            initializer="random_normal",
            trainable=True
        )
        
    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b

In [5]:
class MyReLU(layers.Layer):
    def __init__(self):
        super(MyReLU, self).__init__()
    
    def call(self, x):
        return tf.math.maximum(x, 0)

In [6]:
class MyModel(keras.Model): # model.fit, mode.evaluate, model.predict
    def __init__(self, num_classes=10):
        super(MyModel, self).__init__()
        self.dense1 = Dense(64)
        self.dense2 = Dense(num_classes)
        self.relu = MyReLU()
        
        # self.dense1 = layers.Dense(64)
        # self.dense3 = layers.Dense(num_classes)
        
    def call(self, x):
        x = self.relu(self.dense1(x))
        return self.dense2(x)

In [7]:
model = MyModel()
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(),
    metrics=["accuracy"]
)

In [14]:
model.fit(x_train, y_train, batch_size=32, epochs=2, verbose=1)
print(model.metrics_names)
model.evaluate(x_test, y_test, batch_size=32, verbose=1)

Epoch 1/2
1875/1875 [==============================] - 13s 7ms/step - loss: 0.0843 - accuracy: 0.9751
Epoch 2/2
1875/1875 [==============================] - 22s 12ms/step - loss: 0.0661 - accuracy: 0.9801
['loss', 'accuracy']
313/313 [==============================] - 2s 8ms/step - loss: 0.0886 - accuracy: 0.9747


[0.08858636766672134, 0.9746999740600586]